# Applied Seismology, GEOS 626, University of Alaska Fairbanks

- script hw_sumatraB_hf.ipynb
- template script for analyzing the direct arrival waveforms from Sumatra:
- channel BHZ, duration of up to 5 hours

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import os
import sys
import warnings

from obspy import Stream
from obspy.core import read
from obspy.core import UTCDateTime
from obspy.geodetics import kilometers2degrees, gps2dist_azimuth, degrees2kilometers

from scipy.signal import hilbert

from pysep import Pysep
from pysep.recsec import plotw_rs

from lib_seis import smooth
from lib_seis import station_map_and_table

In [ ]:
# script settings

warnings.filterwarnings('ignore')
plt.rcParams['figure.figsize'] = 10, 5
plt.rcParams['lines.linewidth'] = 1

In [ ]:
# download BHZ data - response removed to displacement

download  = dict( client                       = 'IRIS',
                  origin_time                  = UTCDateTime("2004,12,26,00,58,53"),
                  event_latitude               = 3.09,
                  event_longitude              = 94.26,
                  event_depth_km               = 30,
                  event_magnitude              = 9.1,
                  networks                     = 'G,II,IU',
                  stations                     = '*',
                  locations                    = '*',
                  channels                     = 'BHZ',
                  remove_clipped               = True,
                  remove_insufficient_length   = True,
                  remove_response              = True,
                  output_unit                  = 'DISP',
                  detrend                      = True,
                  demean                       = True,
                  pre_filt                     = None,
                  water_level                  = None,
                  mindistance                  = 0.0 * np.pi/180 * 6371,
                  maxdistance                  = 40.0 * np.pi/180 * 6371,
                  fill_data_gaps               = 0,
                  gap_fraction                 = 0.03,
                  resample_freq                = 20,
                  seconds_before_ref           = 1 * 60 * 60,
                  seconds_after_ref            = 4 * 60 * 60,
                  taup_model                   = 'ak135',
                  log_level                    = 'DEBUG',
                  write_files                  = 'sac',
                  plot_files                   = 'map',
                  output_dir                   = 'datawf',
                  overwrite                    = False,
                  overwrite_event_tag          = 'sumatra_BHZ_disp')

In [ ]:
# download data

data_dir  = f'{download["output_dir"]}/{download["overwrite_event_tag"]}'
sac_dir   = f'{data_dir}/SAC'
overwrite = f'{download["overwrite"]}'

if (not os.path.isdir(data_dir)) or (overwrite == 'True'):
    ps = Pysep(**download)
    ps.run()
else:
    if os.path.isdir(sac_dir):
        print('SAC directory already exists, proceeding to the next code cell')
    else:
        sys.exit('Data directory exists but with no SAC directory.\n\
            Remove data directory and rerun download.')

In [ ]:
# read downloaded raw BHZ data

st       = read(f'{sac_dir}/*.sac', header=None)

print('Details of downloaded waveforms - \n')
print(st.__str__(extended=True))
print()

In [ ]:
station_map_and_table(st,event_lat=download['event_latitude'],event_lon=download['event_longitude'])

In [ ]:
# OPTION A: PLOT ALL WAVEFORMS DOWNLOADED BY PYSEP
# plot a record section using Pyseps's record section plotting tool
# change the following parameters as needed

plot_info = dict( pysep_path                   = sac_dir,
                  sort_by                      = 'distance',
                  scale_by                     = 'normalize',
                  time_shift_s                 = None,
                  min_period_s                 = 1,
                  max_period_s                 = 10,
                  xlim_s                       = [-50, 7200],
                  preprocess                   = 'st',
                  max_traces_per_rs            = 40,
                  distance_units               = 'deg',
                  tmarks                       = [0],
                  save                         = '',
                  log_level                    = 'CRITICAL')

plotw_rs(**plot_info)

In [ ]:
# choose subset of waveforms

picked_waveforms = ['IU.TATO.00.BHZ',
                    'II.PALK.00.BHZ',
                    'II.COCO.00.BHZ']

stsub = Stream()

for i, waveform_id in enumerate(picked_waveforms):
    stsub = stsub + st.select(id=waveform_id)

In [ ]:
station_map_and_table(stsub,event_lat=download['event_latitude'],event_lon=download['event_longitude'])

In [ ]:
# OPTION B: PLOT SUBSET OF WAVEFORMS

plot_info = dict( st                           = stsub,
                  sort_by                      = 'distance',
                  scale_by                     = 'normalize',
                  time_shift_s                 = None,
                  min_period_s                 = 1,
                  max_period_s                 = 10,
                  xlim_s                       = [-50, 7200],
                  preprocess                   = 'st',
                  max_traces_per_rs            = 40,
                  distance_units               = 'deg',
                  tmarks                       = [0],
                  save                         = '',
                  log_level                    = 'CRITICAL')

plotw_rs(**plot_info)